## Packages and Libraries

In [ ]:
!pip install jedi
!pip install snntorch
!pip install brevitas
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import snntorch as snn
from brevitas.nn import QuantLinear
from brevitas.nn import QuantLinear
from brevitas.quant import Int8WeightPerTensorFixedPoint
from brevitas.core.quant import QuantType
from brevitas.nn import QuantLinear
from snntorch import functional as SF
from snntorch import functional as SF
import brevitas.nn as qnn

import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


## Quantization

In [ ]:
import torch
import math

class StateQuant(torch.autograd.Function):
    """Wrapper function for state quantization."""

    @staticmethod
    def forward(ctx, input_, levels):
        device = input_.device
        levels = levels.to(device)
        size = input_.size()
        input_ = input_.flatten()

        # Broadcast levels along a new dimension equal to the number of levels
        input_expanded = input_.unsqueeze(-1).repeat(1, levels.size(0))

        # Find the closest valid quantization state
        differences = torch.abs(levels - input_expanded)
        idx_match = torch.argmin(differences, dim=-1)
        quant_tensor = levels[idx_match]

        return quant_tensor.reshape(size)

    @staticmethod
    def backward(ctx, grad_output):
        # Straight Through Estimator (STE)
        grad_input = grad_output.clone()
        return grad_input, None

def state_quant(num_bits=8, uniform=True, thr_centered=True, threshold=1,
                lower_limit=0, upper_limit=0.2, multiplier=None):
    """Generate a quantization function with specified parameters."""
    num_levels = 2 ** num_bits  # Total number of quantization levels

    if uniform:
        # Uniform quantization
        levels = torch.linspace(
            -threshold * (1 + lower_limit),
            threshold * (1 + upper_limit),
            num_levels
        )
    else:
        # Non-uniform quantization
        if multiplier is None:
            # Default values based on number of bits
            multiplier = 0.05 + 0.9 * (num_bits - 1) / 15

        if thr_centered:
            # Centered around threshold
            max_val = threshold * (1 + upper_limit)
            min_val = -threshold * (1 + lower_limit)
        else:
            # Centered around zero
            max_val = threshold + threshold * upper_limit
            min_val = -threshold - threshold * lower_limit

        range_vals = torch.logspace(
            start=-multiplier * (num_levels - 1),
            end=0,
            steps=num_levels,
            base=10
        )
        levels = min_val + range_vals * (max_val - min_val)

    def inner(x):
        return StateQuant.apply(x, levels)

    return inner

# Example Usage
if __name__ == "__main__":
    num_bits = 5
    q_func = state_quant(num_bits=num_bits, uniform=True, threshold=5)

    # Sample input tensor
    x = torch.rand(10, 10) * 10 - 5  # Random values between -5 and 5
    quant_x = q_func(x)



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the quantization function for state variables
quant_func = state_quant(num_bits=6, uniform=True, threshold=1)

## Weight Constraints

In [ ]:
# Function to apply constraints on weights
def apply_weight_constraints(model):
    # Constraint: Positive integer weights and specific connections set to zero
    with torch.no_grad():
        # Constrain the first layer weights to be positive and integer
        # model.fc1.weight.data = torch.clamp(model.fc1.weight.data, min=0)
        # model.fc1.weight.data = model.fc1.weight.data.round()

        # Apply specific constraints to the first layer weights
        model.fc1.weight.data[1, 0] = 0  # input0 -> neuron1
        model.fc1.weight.data[2, 0] = 0  # input0 -> neuron2
        model.fc1.weight.data[0, 1] = 0  # input1 -> neuron0
        model.fc1.weight.data[2, 1] = 0  # input1 -> neuron2
        model.fc1.weight.data[0, 2] = 0  # input2 -> neuron0
        model.fc1.weight.data[1, 2] = 0  # input2 -> neuron1

        # Set biases to 0
        model.fc1.bias.data.fill_(0)
        model.fc2.bias.data.fill_(0)

        # Constrain the second layer weights to be positive and integer
        # model.fc2.weight.data = torch.clamp(model.fc2.weight.data, min=0)
        # # model.fc2.weight.data = model.fc2.weight.data.round()

## Gray Code Dataset

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the Gray code sequence
gray_codes = [
    [0, 0, 0],  # 0
    [0, 0, 1],  # 1
    [0, 1, 1],  # 2
    [0, 1, 0],  # 3
    [1, 1, 0],  # 4
    [1, 1, 1],  # 5
    [1, 0, 1],  # 6
    [1, 0, 0],   # 7
    [0, 0, 0]  # 0 (looping back)
]


def get_gray_code_index(tensor_vector):

    if tensor_vector.dim() == 1:  # Single vector case
        vector_list = tensor_vector.tolist()  # Convert tensor to list
        for i, code in enumerate(gray_codes):
            if code == vector_list:
                return i
        return -1  # Return -1 if not found

    elif tensor_vector.dim() == 2:  # Matrix case (batch of vectors)
        indices = []
        for row in tensor_vector:
            vector_list = row.tolist()  # Convert each row (vector) to list
            found = False
            for i, code in enumerate(gray_codes):
                if code == vector_list:
                    indices.append(i)
                    found = True
                    break
            if not found:
                indices.append(-1)  # If the vector is not found, append -1
        return indices  # Return tensor of indices

# Function to get Gray codes for a list of indices
def get_gray_codes_from_indices(indices):
    # Create a list to hold the Gray codes corresponding to the indices
    gray_code_list = [gray_codes[i] for i in indices]

    # Convert the list of Gray codes to a tensor
    gray_code_tensor = torch.tensor(gray_code_list)

    return gray_code_tensor

max_index = len(gray_codes)-1  # Total number of Gray codes (9 in this case)
# Function to increment indices by 1, wrapping around if necessary
def increment_indices(indices, max_index):
    incremented_indices = [(i + 1) % max_index for i in indices]
    return incremented_indices

def decrement_indices(indices, max_index=8):
    decremented_indices = [(i - 1) % max_index for i in indices]
    return decremented_indices



# Create input-output pairs for training
X = torch.tensor(gray_codes[:-1], dtype=torch.float32)  # Inputs: 000 to 110
y = torch.tensor(gray_codes[1:], dtype=torch.float32)   # Outputs: 001 to 100 (next Gray code)

# Replicate X and y to make a bigger dataset
replication_factor = 100  # Number of times to replicate the dataset
X_replicated = X.repeat((replication_factor, 1))
y_replicated = y.repeat((replication_factor, 1))

# # Shuffle the dataset
# indices = torch.randperm(X_replicated.size(0))
# X_shuffled = X_replicated[indices]
# y_shuffled = y_replicated[indices]

# Create DataLoader
# dataset = TensorDataset(X_shuffled, y_shuffled)
# train_loader = DataLoader(dataset, batch_size=18, shuffle=True)
dataset = TensorDataset(X_replicated, y_replicated)
train_loader = DataLoader(dataset, batch_size=8, shuffle=False)


## SNN: model and training

In [ ]:

# Define the model
class NetSNN(nn.Module):
    def __init__(self, timesteps, hidden, beta, quant_bit_width=8):
        super().__init__()
        self.timesteps = timesteps
        self.hidden = hidden
        self.beta = beta

        # Corrected Quantized linear layers using Brevitas
        self.fc1 = QuantLinear(3, hidden, bias=True, bit_width=quant_bit_width,
                               weight_quant_type=QuantType.INT, weight_bit_width=quant_bit_width)
        # Spiking neuron layers with quantized state variables
        self.lif1 = snn.Leaky(beta=self.beta)

        self.fc2 = QuantLinear(hidden, 3, bias=True, bit_width=quant_bit_width,
                               weight_quant_type=QuantType.INT, weight_bit_width=quant_bit_width)
        self.lif2 = snn.Leaky(beta=self.beta)

    def forward(self, x):
        decay_value=0.01
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        spk_recording = []
        decay_value1 =0.01
        indices = get_gray_code_index(x)
        for step in range(self.timesteps):
            x=get_gray_codes_from_indices(indices)
            x=x.float()
            cur1 = self.fc1(x)
            if step != 0:
              mem1 = torch.where(mem1 < decay_value, cur1, mem1 - decay_value)
              self.lif1.mem = mem1
            spk1, mem1 = self.lif1(cur1, mem1)

            cur2 = self.fc2(spk1)
            if step != 0:
              mem2 = torch.where(mem2 < decay_value, cur2, mem2 - decay_value)
              self.lif2.mem = mem2
            spk2, mem2 = self.lif2(cur2, mem2)
            spk_recording.append(spk2)
            # if step % 2 == 0:
            # indices=increment_indices(indices, max_index)
            indices=decrement_indices(indices)
            # print(f"mem1: {indices }")
        return torch.stack(spk_recording)

# Instantiate the model
num_steps = 4  # Number of time steps
hidden = 3     # Number of hidden neurons
beta = 0.999     # Membrane potential decay rate
model = NetSNN(timesteps=num_steps, hidden=hidden, beta=beta).to(device)

# Optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.005)
loss_function = nn.MSELoss()  # Use MSE as it's suitable for regression-type tasks like this

best_loss = float('inf')  # Set initial best loss to infinity
best_model_path = "best_model.pth"  # File path to save the best model

epoch_loss = 100


# Training loop
num_epochs = 100
accuracy=0
while accuracy<100:
  model.train()
  for epoch in range(num_epochs):
      for data, targets in train_loader:
          data = data.to(device)
          targets = targets.to(device)

          # apply_weight_constraints(model)

          spk = model(data)
          # Take the mean across the time steps
          output = spk[-1] #.mean(dim=0)
          loss_val = loss_function(output, targets)  # apply loss
          optimizer.zero_grad()  # zero out gradients
          loss_val.backward()  # calculate gradients
          optimizer.step()  # update weights

          # Apply constraints after each update
          apply_weight_constraints(model)

      # Save the model if the current epoch's loss is lower than the best loss
      if epoch_loss > loss_val.item() or epoch==0:
        epoch_loss = loss_val.item()
        torch.save(model.state_dict(), best_model_path)
        print(f"Epoch {epoch}: Saving model with loss {epoch_loss:.4f}")

      if epoch % 10 == 0:
          print(f'Epoch {epoch}, Loss: {loss_val.item()}')

  # Testing and Evaluation loop
  model.load_state_dict(torch.load(best_model_path))
  model.eval()
  correct_predictions = 0
  total_predictions = 0

  with torch.no_grad():
      for data, targets in train_loader:
          data = data.to(device)
          targets = targets.to(device)

          spk = model(data)
          # Take the mean across the time steps
          output = spk[-1] #.mean(dim=0)
          predicted = output #.round()  # Round to the nearest integer (0 or 1)

          correct_predictions += (predicted == targets).all(dim=1).sum().item()
          total_predictions += targets.size(0)

          # print(f"Input: {data.cpu().numpy()}, Predicted: {predicted.cpu().numpy()}, Target: {targets.cpu().numpy()}")

  accuracy = correct_predictions / total_predictions * 100
  print(f"Accuracy: {accuracy:.2f}%")



Epoch 0: Saving model with loss 0.3750
Epoch 0, Loss: 0.375
Epoch 1: Saving model with loss 0.2083
Epoch 7: Saving model with loss 0.1667
Epoch 10, Loss: 0.1666666716337204
Epoch 15: Saving model with loss 0.1250
Epoch 20, Loss: 0.125
Epoch 24: Saving model with loss 0.0833
Epoch 30, Loss: 0.0833333358168602
Epoch 40, Loss: 0.0833333358168602
Epoch 50, Loss: 0.0833333358168602
Epoch 60, Loss: 0.0833333358168602
Epoch 70, Loss: 0.0833333358168602
Epoch 80, Loss: 0.0833333358168602
Epoch 86: Saving model with loss 0.0000
Epoch 90, Loss: 0.0


<ipython-input-7-79ad095d25f4>:178: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


Accuracy: 100.00%


## Model Evaluation

In [ ]:
# Testing and Evaluation loop
model.load_state_dict(torch.load(best_model_path))
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for data, targets in train_loader:
        data = data.to(device)
        targets = targets.to(device)

        spk = model(data)
        # Take the mean across the time steps
        output = spk[-1] #.mean(dim=0)
        predicted = output #.round()  # Round to the nearest integer (0 or 1)

        correct_predictions += (predicted == targets).all(dim=1).sum().item()
        total_predictions += targets.size(0)

        # print(f"Input: {data.cpu().numpy()}, Predicted: {predicted.cpu().numpy()}, Target: {targets.cpu().numpy()}")

accuracy = correct_predictions / total_predictions * 100
print(f"Accuracy: {accuracy:.2f}%")

<ipython-input-8-8cb227cb8b47>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


Accuracy: 100.00%


In [ ]:
# Print the properties of the spiking neurons in the model
def print_snn_properties(model):
    for name, layer in model.named_modules():
        if isinstance(layer, snn.Leaky):
            print(f"Properties of {name}:")
            print(f"  - Leaky (beta): {layer.beta}")  # Membrane potential decay rate
            # Check if threshold and refractory period attributes exist
            threshold = getattr(layer, 'threshold', None)
            refractory_period = getattr(layer, 'refractory', None)
            if threshold is not None:
                print(f"  - Threshold: {threshold}")
            else:
                print(f"  - Threshold: Not specified in this layer.")

            print()

# Call the function to print properties of the spiking neurons
print_snn_properties(model)


Properties of lif1:
  - Leaky (beta): 0.9990000128746033
  - Threshold: 1.0

Properties of lif2:
  - Leaky (beta): 0.9990000128746033
  - Threshold: 1.0



In [ ]:
# Create input-output pairs for the 8 combinations of the Gray code + 1
X = torch.tensor(gray_codes[:-1], dtype=torch.float32)  # Inputs
y = torch.tensor(gray_codes[1:], dtype=torch.float32)   # Outputs

# Create DataLoader without repetition and no shuffle
gray_code_test_dataset = TensorDataset(X, y)
gray_code_test_loader = DataLoader(gray_code_test_dataset, batch_size=1, shuffle=False)

# Initialize counters for correct and incorrect predictions
correct_predictions = 0
incorrect_predictions = 0

# Evaluate the model on all 8 unique combinations of the Gray code
with torch.no_grad():
    for data, targets in gray_code_test_loader:
        data, targets = data.to(device), targets.to(device)  # Move data and targets to device

        spk = model(data)
        output = spk[-1] # Mean across time steps and round to nearest integer

        # Check if the prediction matches the target
        if (output == targets).all():
            correct_predictions += 1
        else:
            incorrect_predictions += 1

        # Print input, predicted, and target values
        print(f"Input: {data.cpu().numpy()}, Predicted: {output.cpu().numpy()}, Target: {targets.cpu().numpy()}")

# Calculate overall accuracy
total_predictions = correct_predictions + incorrect_predictions
accuracy = (correct_predictions / total_predictions) * 100

# Print summary of results
print(f"\nCorrect Predictions: {correct_predictions}")
print(f"Incorrect Predictions: {incorrect_predictions}")
print(f"Overall Accuracy: {accuracy:.2f}%")

Input: [[0. 0. 0.]], Predicted: [[0. 0. 1.]], Target: [[0. 0. 1.]]
Input: [[0. 0. 1.]], Predicted: [[0. 1. 1.]], Target: [[0. 1. 1.]]
Input: [[0. 1. 1.]], Predicted: [[0. 1. 0.]], Target: [[0. 1. 0.]]
Input: [[0. 1. 0.]], Predicted: [[1. 1. 0.]], Target: [[1. 1. 0.]]
Input: [[1. 1. 0.]], Predicted: [[1. 1. 1.]], Target: [[1. 1. 1.]]
Input: [[1. 1. 1.]], Predicted: [[1. 0. 1.]], Target: [[1. 0. 1.]]
Input: [[1. 0. 1.]], Predicted: [[1. 0. 0.]], Target: [[1. 0. 0.]]
Input: [[1. 0. 0.]], Predicted: [[0. 0. 0.]], Target: [[0. 0. 0.]]

Correct Predictions: 8
Incorrect Predictions: 0
Overall Accuracy: 100.00%


## Quantized data and parameters

In [ ]:
# Print the actual weights of the first layer
print("First layer weights:\n", model.fc1.weight.data)

# Print quantized weight scales for the first layer
print("\nFirst layer quantization scale:\n", model.fc1.quant_weight_scale())

# Check the quantized weights range and values
quantized_weights_fc1 = model.fc1.weight.data / model.fc1.quant_weight_scale()
print("\nQuantized values of first layer (scaled to integer range):\n", quantized_weights_fc1.int())

# Repeat the process for the second layer
print("\nSecond layer weights:\n", model.fc2.weight.data)
print("\nSecond layer quantization scale:\n", model.fc2.quant_weight_scale())
quantized_weights_fc2 = model.fc2.weight.data / model.fc2.quant_weight_scale()
print("\nQuantized values of second layer (scaled to integer range):\n", quantized_weights_fc2.int())


First layer weights:
 tensor([[1.3052, 0.0000, 0.0000],
        [0.0000, 1.1449, 0.0000],
        [0.0000, 0.0000, 2.5882]])

First layer quantization scale:
 tensor(0.0204, grad_fn=<DivBackward0>)

Quantized values of first layer (scaled to integer range):
 tensor([[ 64,   0,   0],
        [  0,  56,   0],
        [  0,   0, 127]], dtype=torch.int32)

Second layer weights:
 tensor([[-1.8717e+00,  2.0085e+00,  1.4848e-01],
        [-1.0072e-01, -1.1686e+00,  1.2330e+00],
        [ 7.3247e-05,  7.2581e-03,  6.7786e-01]])

Second layer quantization scale:
 tensor(0.0158, grad_fn=<DivBackward0>)

Quantized values of second layer (scaled to integer range):
 tensor([[-118,  127,    9],
        [  -6,  -73,   77],
        [   0,    0,   42]], dtype=torch.int32)


In [ ]:
import torch

def tensor_to_hex_and_bits_signed_8bit(tensor):
    # First, we clamp the values to the 8-bit signed integer range (-128 to 127)
    tensor = torch.clamp(tensor, -128, 127).int()

    # Initialize lists to store hexadecimal values and bit arrays
    hex_values = []
    bit_arrays = []

    # Iterate through the tensor, converting each value to 8-bit signed hex and bit array
    for value in tensor.view(-1):  # Flatten the tensor for easy iteration
        if value < 0:
            # Convert negative number to its two's complement hex and bit representation
            two_complement_value = (1 << 8) + value.item()  # 8-bit two's complement
        else:
            # Positive value (no two's complement required)
            two_complement_value = value.item()

        # Convert to hex, removing '0x' prefix and padding to 2 characters
        hex_value = hex(two_complement_value)[2:].zfill(2)
        hex_values.append(hex_value)

        # Convert to 8-bit binary string and store the result as a string (instead of a list)
        bit_array = bin(two_complement_value)[2:].zfill(8)  # 8-bit binary
        bit_arrays.append(bit_array)

    return hex_values, bit_arrays

# Example usage:
tensor = torch.tensor([10, -5, 127, -128, 45, -12])
hex_result, bit_result = tensor_to_hex_and_bits_signed_8bit(tensor)

print("Hexadecimal (2's complement) 8-bit signed values:", hex_result)
print("Bit arrays (2's complement) 8-bit signed values:")
for bits in bit_result:
    print(bits)


Hexadecimal (2's complement) 8-bit signed values: ['0a', 'fb', '7f', '80', '2d', 'f4']
Bit arrays (2's complement) 8-bit signed values:
00001010
11111011
01111111
10000000
00101101
11110100


In [ ]:
import numpy as np

def normalize_tensor(tensor,tensor_min,tensor_max, a=-128, b=127):

    # Normalize the tensor between a and b (e.g., -127 and 128)
    tensor_normalized = (b - a) * (tensor - tensor_min) / (tensor_max - tensor_min) + a

    return tensor_normalized

def quantize_tensor(tensor, n_bits,tensor_min,tensor_max):
    # Calculate the range for n-bit signed two's complement
    min_val = -(2 ** (n_bits - 1))
    max_val = (2 ** (n_bits - 1)) - 1

    tensor_scaled=normalize_tensor(tensor,tensor_min,tensor_max,-max_val, max_val)

    # Quantize to nearest integer
    tensor_quantized = np.round(tensor_scaled)


    # dequantize back to the original range
    tensor_dequantized = ((tensor_quantized + max_val) / (max_val + max_val)) * (tensor_max - tensor_min) + tensor_min

    return tensor_quantized, tensor_dequantized

# Example usage
# tensor = np.array([[0.5, 1.2, -0.3], [3.4, -2.1, 0.0]], dtype=np.float32)
# n_bits = 8  # Example: 8-bit quantization

# # Find the maximum absolute value
# max_abs_value = np.max(np.abs(tensor))

# quantized_tensor, dequantized_tensor = quantize_tensor(tensor, n_bits,-max_abs_value,max_abs_value)

# print("Original Tensor:")
# print(tensor)
# print("\nQuantized Tensor:")
# print(quantized_tensor)
# print("\nDequantized Tensor (approximation of the original):")
# print(dequantized_tensor)

# Get absolute values of both tensors
overall_max = torch.max(torch.max(torch.abs(model.fc1.weight.data)),torch.max(torch.abs(model.fc2.weight.data)))

n_bits = 6  # Example: 8-bit quantization

# Print the weights of the first layer
print("--------------------------------------------\nFirst layer weights:\n--------------------------------------------")
print("Original first layer weights:\n", model.fc1.weight.data)
quantized_tensor_1, dequantized_tensor_1 = quantize_tensor(model.fc1.weight.data, n_bits,-overall_max,overall_max)
# Print quantized weight scales for the first layer
print("\nFirst layer quantization:\n", quantized_tensor_1)
print("\nDequantized First layer (approximation of the original):")
print(dequantized_tensor_1)
print("\nhex_result - bit_result:")
hex_result, bit_result = tensor_to_hex_and_bits_signed_8bit(quantized_tensor_1)
print("Hexadecimal (2's complement) 8-bit signed values:", hex_result)
print("Bit arrays (2's complement) 8-bit signed values:")
for bits in bit_result:
    print(bits)

# Print the weights of the second layer
print("\n\n--------------------------------------------\nSecond layer weights:\n--------------------------------------------")
print("Original second layer weights:\n", model.fc2.weight.data)
quantized_tensor_2, dequantized_tensor_2 = quantize_tensor(model.fc2.weight.data, n_bits,-overall_max,overall_max)
# Print quantized weight scales for the first layer
print("\nFirst layer quantization:\n", quantized_tensor_2)
print("\nDequantized First layer (approximation of the original):")
print(dequantized_tensor_2)
print("\nhex_result - bit_result:")
hex_result, bit_result = tensor_to_hex_and_bits_signed_8bit(quantized_tensor_2)
print("Hexadecimal (2's complement) 8-bit signed values:", hex_result)
print("Bit arrays (2's complement) 8-bit signed values:")
for bits in bit_result:
    print(bits)


decay_value=0.01
# Print the quantized decay and threshold
print("\n\n--------------------------------------------\nDecay and Threshold:\n--------------------------------------------")
print("Decay:\n", decay_value)
quantized_decay_value, dequantized_decay_value = quantize_tensor(decay_value, n_bits,-overall_max,overall_max)
# Print quantized weight scales for the first layer
print("\nquantized_decay_value:\n", quantized_decay_value)
print("\ndequantized_decay_value (approximation of the original):")
print(dequantized_decay_value)
print("\nhex_result - bit_result:")
hex_result, bit_result = tensor_to_hex_and_bits_signed_8bit(quantized_decay_value)
print("Hexadecimal (2's complement) 8-bit signed values:", hex_result)
print("Bit arrays (2's complement) 8-bit signed values:")
for bits in bit_result:
    print(bits)


threshold=1
print("Threshold:\n", threshold)
quantized_threshold, dequantized_threshold = quantize_tensor(threshold, n_bits,-overall_max,overall_max)
# Print quantized weight scales for the first layer
print("\quantized_threshold:\n", quantized_threshold)
print("\dequantized_threshold (approximation of the original):")
print(dequantized_threshold)
print("\nhex_result - bit_result:")
hex_result, bit_result = tensor_to_hex_and_bits_signed_8bit(quantized_threshold)
print("Hexadecimal (2's complement) 8-bit signed values:", hex_result)
print("Bit arrays (2's complement) 8-bit signed values:")
for bits in bit_result:
    print(bits)

--------------------------------------------
First layer weights:
--------------------------------------------
Original first layer weights:
 tensor([[1.3052, 0.0000, 0.0000],
        [0.0000, 1.1449, 0.0000],
        [0.0000, 0.0000, 2.5882]])

First layer quantization:
 tensor([[16.,  0.,  0.],
        [ 0., 14.,  0.],
        [ 0.,  0., 31.]])

Dequantized First layer (approximation of the original):
tensor([[1.3358, 0.0000, 0.0000],
        [0.0000, 1.1689, 0.0000],
        [0.0000, 0.0000, 2.5882]])

hex_result - bit_result:
Hexadecimal (2's complement) 8-bit signed values: ['10', '00', '00', '00', '0e', '00', '00', '00', '1f']
Bit arrays (2's complement) 8-bit signed values:
00010000
00000000
00000000
00000000
00001110
00000000
00000000
00000000
00011111


--------------------------------------------
Second layer weights:
--------------------------------------------
Original second layer weights:
 tensor([[-1.8717e+00,  2.0085e+00,  1.4848e-01],
        [-1.0072e-01, -1.1686e+00,